In [14]:
import re

In [66]:
with open('structure_tree.txt', 'r') as f:
    lines = f.readlines()
    lines = [l.rstrip() for l in lines if len(l) > 0]
    
    current = {}
    structure_list = {}
    
    for l in lines:
        if l.startswith('          - '):
            level = 3
            text = l.strip()[2:]
        elif l.startswith('     - '):
            level = 2
            text = l.strip()[2:]
        elif l.startswith('     '):
            level = 1
            text = l.strip()
        else:
            level = 0
            text = l.strip()
            
        if len(text) == 0:
            continue
        
        try:
            res = re.findall('^(.+?)(:\s?(.*))?$', text)[0]
        except:
            print 'text ERROR:', text
            raise
            
        name, _, desc = res
        
        current[level] = name
        
        try:
            fullname, abbreviation = re.findall('^(.*),\s?(.*?)$', name)[0]
        except:
            fullname = name
            abbreviation = ''
        
        parent = current[level-1] if level > 0 else None
        
        structure_list[name] = {'level': level, 
               'fullname': fullname, 
               'abbr': abbreviation, 
               'description': desc,
              'parent': parent,
                'children': []}
        
        if parent is not None:
            structure_list[parent]['children'].append(name)
        

In [70]:
for name, s in structure_list.iteritems():
    if s['level'] == 2:
        print name

raphe magnus, RMg
medial inferior olive nucleus, IOM
intermediate gray, InG
nucleus of Roller/sublingual nucleus, Ro
ventrolateral
paramedian reticular nucleus, PMn
caudal pontine reticular nucleus, PnC
dorsal cochlear nucleus, DC
nucleus of the trapezoid body, Tz
lateral, LVe
external, ECIC
medial
nucleus of the optic tract, OT
dorsal (pericentral) cortex of the inferior colliculus, DCIC
reticulotegmental nucleus, PtTg
lateral
parvicellular
superior, SVe
central group
brachium of the inferior colliculus, bic
ventral cochlear nucleus, VC
prepositus nucleus, Pr
rostral
medial, MVe
nucleus of pretectal area
Superficial gray layer, SuG
pontine central grey, CGPn
dorsolateral
apical
substantia nigra, compact part, SNC
commissure of inferior colliculus, cic
intermediate white, InWh
Superior Olive, SO
parvocellular, RPC
central, CIC
Intercalated Nucleus, In
intermediate
deep gray, DpG
raphe pallidus, RPa
median raphe nucleus, MnR
paramedian raphe nucleus, PMnR
principle inferior olive nucleu